# **Examples for Langchains and Azure OpenAI**

In [1]:
# import os module & the OpenAI Python library for calling the OpenAI API 
import os
import openai
import json
from os import path
import numpy as np

## Reference

* https://python.langchain.com/en/latest/modules/models/llms/integrations/azure_openai_example.html

**Using LLM Models**
1) Completion API with Text in Text Out Mode

In [10]:
%pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import openai
from langchain.llms import AzureOpenAI
from langchain.embeddings import OpenAIEmbeddings


In [7]:
# Configure OpenAI API
# Load config values
# Setting up the deployment name
gpt_model_name = "gpt-4"
print(gpt_model_name)

# This is set to `azure`
openai.api_type = "azure"

# The API key for your Azure OpenAI resource.
openai.api_key = os.getenv("OPENAI_API_KEY")
#API_KEY = openai.api_key
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai.api_base = "<your URL>"

# The Azure OPENAI API version.
openai.api_version = "2023-03-15-preview"

gpt-4


In [13]:
# Connecting to Azure OpenAI
llm = AzureOpenAI(
    deployment_name="text-davinci-003" ,
    openai_api_key= API_KEY
    )

In [14]:
joke = llm("Tell me a dad joke")
print(joke)



Q: What did the baby corn say to the mama corn?
A: Where's Popcorn?


**ChatAPI** for using Chat GPT Model 


In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chat = ChatOpenAI(model='gpt-35-turbo',
                  deployment_id = "gpt-35-turbo",
                  temperature=0.9,
                  openai_api_key=API_KEY)

response = chat(
    [
        SystemMessage(content="You are an AI assistant that helps a user with marketing content. Keep responses short and under 5 sentences."),
        HumanMessage(content="Write a review of Electric vehicles in US")
    ]
)

response

WARNING! deployment_id is not default parameter.
                    deployment_id was transferred to model_kwargs.
                    Please confirm that deployment_id is what you intended.


AIMessage(content='Electric vehicles are becoming increasingly popular in the US market. They are eco-friendly, offer efficiency, and are attractive to consumers seeking cost savings. Federal and state incentives for EV purchases have helped drive adoption rates. However, concerns over range anxiety and the availability of charging infrastructure remain challenges for widespread adoption. Overall, the future looks bright for electric vehicles in the US.', additional_kwargs={}, example=False)

**Embeddings**



In [23]:
from langchain.embeddings import OpenAIEmbeddings

openai_embed = OpenAIEmbeddings(model='text-embedding-ada-002',
                                openai_api_key=API_KEY)

query_embeddings = openai_embed.embed_query("The sky is blue") 

print(type(query_embeddings))
print(len(query_embeddings))
print(query_embeddings[:5])

<class 'list'>
1536
[0.009326309897005558, -0.0006310197059065104, 0.006163125857710838, -0.015400114469230175, -0.013527460396289825]


**Using Prompt Templates** for use input and capturing any variables / non static informations

In [32]:
from langchain.prompts import PromptTemplate

# Define a prompt template with placeholders for input variables.
prompt_template = PromptTemplate(
        template = 'Write a small advertisement a new {product} and its description {product_desc}?',
        input_variables = ['product', 'product_desc']
)

# Create the final prompt by filling in the defined variables.
final_prompt = prompt_template.format(product='Headphones',  product_desc='Best headphones')
print("FINAL PROMPT: " + final_prompt)

response = llm(final_prompt) 
print(response)

FINAL PROMPT: Write a small advertisement a new Headphones and its description Best headphones?
 We've got them! 

Introducing the latest in sound technology, our new headphones make listening to music a truly immersive experience. With crystal-clear audio, deep bass, and noise-cancelling capabilities, these headphones will take your music experience to the next level. They come with a long-lasting battery life and a comfortable, lightweight design, so you can enjoy your tunes all day without having to worry about a recharge. Plus, they look great too, with a sleek, modern design that will look good no matter where you take them. Get ready to experience music like never before, with our amazing new headphones!


In [34]:
print(llm(prompt_template.format(product ='Electric Vehicle', product_desc='Iceland')))



Experience the thrill of a new electric vehicle in Iceland! Our electric vehicle is perfect for exploring the breathtaking scenery of Iceland. With its powerful motor, you will be able to navigate the rugged terrain of the landscape with ease. Enjoy a smooth, quiet ride as you take in the natural beauty of Iceland. Our electric vehicle offers superior performance with a range of up to 300 kilometers. It has a modern design, stylish looks, and is loaded with features. Don't miss out on this amazing opportunity to explore Iceland in the comfort and style of a new electric vehicle.


**Structured Output Parser**

In [35]:
# 1. Define the schema

from langchain.output_parsers import StructuredOutputParser, ResponseSchema

response_schemas = [
    ResponseSchema(name="product", description="This is a product"),
    ResponseSchema(name="product_desc", description="This is a short description of the product")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [36]:
# 2. Generates output format instructions which can be included in your prompt.

format_instructions = output_parser.get_format_instructions()
print (format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"product": string  // This is a product
	"product_desc": string  // This is a short description of the product
}
```


In [37]:
# 3. Incorporate into final prompt.

template = '''
You will be given a product by the user.
Please provide a imaginary, creative and non offensive product name and a short description of the product.

{format_instructions}

% PRODUCT:
{product}

YOUR RESPONSE:
'''

prompt_template = PromptTemplate(
    input_variables=["product"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)

prompt = prompt_template.format(product="Iphone")
print('FINAL PROMPT:')
print(prompt)

FINAL PROMPT:

You will be given a product by the user.
Please provide a imaginary, creative and non offensive product name and a short description of the product.

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"product": string  // This is a product
	"product_desc": string  // This is a short description of the product
}
```

% PRODUCT:
Iphone

YOUR RESPONSE:



In [38]:
# 4. LLM now outputs an easily parsable response.

response = llm(prompt)
print(response)

```json
{
	"product": "Iphone", 
	"product_desc": "A sleek and stylish smartphone that offers cutting edge features and the latest technology." 
}
```


In [39]:
# 5. Output parser can extract named fields in the schema.

output_parser.parse(response)['product_desc']

'A sleek and stylish smartphone that offers cutting edge features and the latest technology.'

**Sequential Chains**

In [40]:
# Chain component 1: 

from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain

template1 = """You will be given an error from the user. Please provide a possible solution.

% ERROR
{error}

YOUR RESPONSE:
"""
prompt_template1 = PromptTemplate(input_variables=["error"], template=template1)

# LLMChain is a simple chain which takes a prompt template, formats with user input and returns LLM output.
error_chain = LLMChain(llm=llm, prompt=prompt_template1)

In [41]:
# Chain component 2: 

template2 = """Given the error message, provide a short solution for fixing it.

% MESSAGE
{message}

YOUR RESPONSE:
"""
prompt_template2 = PromptTemplate(input_variables=["message"], template=template2)

message_chain = LLMChain(llm=llm, prompt=prompt_template2)

In [45]:
overall_chain = SimpleSequentialChain(
    chains=[error_chain, message_chain],
    verbose=True)
chain_output = overall_chain.run("ZeroDivisionError")



> Entering new SimpleSequentialChain chain...


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Creates a completion for the provided prompt, parameters and chosen model. Operation under Azure OpenAI API version 2023-03-15-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 60 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..


This error is caused when a number is divided by zero. To solve this error, make sure that the number being divided is not zero.


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Creates a completion for the provided prompt, parameters and chosen model. Operation under Azure OpenAI API version 2023-03-15-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 56 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Creates a completion for the provided prompt, parameters and chosen model. Operation under Azure OpenAI API version 2023-03-15-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 52 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..
Retrying langc

RateLimitError: Requests to the Creates a completion for the provided prompt, parameters and chosen model. Operation under Azure OpenAI API version 2023-03-15-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 30 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.

In [44]:
print(chain_output)

NameError: name 'chain_output' is not defined

**More Features like Q&A , document loader , text splitting and Agents**


In [46]:
from langchain.document_loaders import GutenbergLoader

# Load a long document.
loader = GutenbergLoader('https://www.gutenberg.org/cache/epub/1515/pg1515.txt')
doc = loader.load()

print(doc[0].page_content[:80].rstrip())
print('Document length in characters:', len(doc[0].page_content))

The Project Gutenberg eBook of The Merchant of Venice, by William Shakespeare
Document length in characters: 154211


In [47]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

# We need to split long documents into smaller chunks to stay under the LLM token limit.
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(doc[0].page_content)
texts[2]

'THE PRINCE OF MOROCCO, suitor to Portia\r\n\n\nTHE PRINCE OF ARRAGON, suitor to Portia\r\n\n\nANTONIO, a merchant of Venice\r\n\n\nBASSANIO, his friend, suitor to Portia\r\n\n\nGRATIANO, friend to Antonio and Bassanio\r\n\n\nSOLANIO, friend to Antonio and Bassanio\r\n\n\nSALARINO, friend to Antonio and Bassanio\r\n\n\nLORENZO, in love with Jessica\r\n\n\nSHYLOCK, a rich Jew\r\n\n\nTUBAL, a Jew, his friend\r\n\n\nLAUNCELET GOBBO, a clown, servant to Shylock\r\n\n\nOLD GOBBO, father to Launcelet\r\n\n\nLEONARDO, servant to Bassanio\r\n\n\nBALTHAZAR, servant to Portia\r\n\n\nSTEPHANO, servant to Portia\r\n\n\nSALERIO, a messenger from Venice\r\n\n\n\r\n\n\nPORTIA, a rich heiress\r\n\n\nNERISSA, her waiting-woman\r\n\n\nJESSICA, daughter to Shylock\r\n\n\n\r\n\n\nMagnificoes of Venice, Officers of the Court of Justice, a Gaoler,\r\n\n\nServants and other Attendants\r\n\n\n\r\n\n\nSCENE: Partly at Venice, and partly at Belmont, the seat of Portia on\r\n\n\nthe Continent\r\n\n\n\r\n\n\n\r\n

**Agents**

In [ ]:
from langchain.utilities import BingSearchAPIWrapper, WikipediaAPIWrapper, PythonREPL

# Bing API usage requires the BING_SUBSCRIPTION_KEY and BING_SEARCH_URL environment variables to be present.
# bingapi = BingSearchAPIWrapper(k=1)
# wikipedia = WikipediaAPIWrapper(top_k_results=1)
# python_repl = PythonREPL()